In [1]:
import json
import os
import re
import csv
from annotationToString import convert_annotations_to_strings

annotations_folder = 'annotations'
final_annotations_file = 'final_annotations.json'

file_path = os.path.join(annotations_folder, final_annotations_file)

with open(file_path, 'r', encoding='utf-8') as f:
    final_data = json.load(f)
f.close()

In [3]:
def dataset_generator(final_data):

    dataset_aux = []
    final_rotulation = []
    item_count = 0
    patent_count = 0

    for index in range(len(final_data["annotations"])):
        annotation = final_data["annotations"][index]

        if annotation is not None and annotation[1]['entities']:
            
            patent_count += 1
            text = annotation[0]
            entities = annotation[1]
            
            match_length = len(re.match(r'\d_\d+_', text).group())
            text = re.sub(r'\d_\d+_', '', text)
            text = re.sub(r'\r', '', text)
            
            rotulation = convert_annotations_to_strings(text, entities, match_length)
            final_rotulation.append(rotulation)

            word_list = re.findall(r'\w+\s?|[^\w]\s?', text, re.UNICODE)
            start = 0
            end = 0
            sentence = 1

            for word in word_list:
                final_word = word.split()[0]
                end = start + len(word)-1
                dataset_aux.append([
                    item_count,
                    final_word,
                    'O',
                    start,
                    end,
                    sentence,
                    patent_count,
                ])
                item_count += 1
                start = start + len(word)
                if(word == '. '):
                    sentence += 1
    index = 0
    patent_num_aux = 1
    for word in dataset_aux:
        patent_num = word[6]
        if(patent_num != patent_num_aux):
            index += 1
        for entity in final_rotulation[index]:
            if(word[1] != '.'):
                if (word[3] == entity[2][0] and word[4] <= entity[2][1]):
                    word[2] = 'B-' + entity[1]
                elif (word[3] >= entity[2][0] and word[4] <= entity[2][1]):
                    word[2] = 'I-' + entity[1]
                    break
        patent_num_aux = patent_num

    dataset = [sublist[:3] + sublist[5:] for sublist in dataset_aux]
    return dataset

In [4]:
def partition_maker(dataset, partition_number):
    dataset_size = len(dataset)
    partition_number_size = dataset_size/partition_number

    count = 0
    partition = 1
    actual_patent = 1
    for data in dataset:
        count += 1
        if(actual_patent != data[4]):
            if(count > partition_number_size):
                count = 0
                partition += 1
        data.append(partition)
        if(partition == 8):
            data.append("validation")
        elif(partition > 8):
            data.append("test")
        else:
            data.append("training")
        actual_patent = data[4]
    return dataset

In [25]:
dataset = dataset_generator(final_data)
dataset = partition_maker(dataset, 10)

for data in dataset:
    actual_partition
    if(data[5]):
        print(data)

[0, 'Fitting', 'O', 1, 1, 1, 'training']
[1, 'room', 'O', 1, 1, 1, 'training']
[2, 'management', 'O', 1, 1, 1, 'training']
[3, 'and', 'O', 1, 1, 1, 'training']
[4, 'occupancy', 'B-SOFTWARE', 1, 1, 1, 'training']
[5, 'monitoring', 'I-SOFTWARE', 1, 1, 1, 'training']
[6, 'system', 'I-SOFTWARE', 1, 1, 1, 'training']
[7, 'for', 'O', 1, 1, 1, 'training']
[8, 'retail', 'O', 1, 1, 1, 'training']
[9, 'operations', 'O', 1, 1, 1, 'training']
[10, 'having', 'O', 1, 1, 1, 'training']
[11, 'fitting', 'O', 1, 1, 1, 'training']
[12, 'rooms', 'O', 1, 1, 1, 'training']
[13, 'wherein', 'O', 1, 1, 1, 'training']
[14, 'the', 'O', 1, 1, 1, 'training']
[15, 'system', 'O', 1, 1, 1, 'training']
[16, 'if', 'O', 1, 1, 1, 'training']
[17, 'capable', 'O', 1, 1, 1, 'training']
[18, 'of', 'O', 1, 1, 1, 'training']
[19, 'notifying', 'B-FUNCIONALIDADE', 1, 1, 1, 'training']
[20, 'staff', 'I-FUNCIONALIDADE', 1, 1, 1, 'training']
[21, 'of', 'O', 1, 1, 1, 'training']
[22, 'fitting', 'O', 1, 1, 1, 'training']
[23, 'room',

In [26]:
headers = ["item", "token", "tag", "sentence", "patent", "partition", "trainingTest"]

with open('final_dataset.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(headers)
    writer.writerows(dataset)